Replicates results from Figure 3 of Burdick, Bouman, Rimon "From Multi-Target Sensory Coverage to Complete Sensory Coverage: An Optimization-Based Robotic Sensory Coverage Approach"

In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import chain, combinations
from spp.convex_sets import Singleton, Ellipsoid
from spp.convex_functions import TwoNorm, SquaredTwoNorm
from spp.graph import GraphOfConvexSets
from spp.shortest_path import ShortestPathProblem

In [ ]:
# convex sets
A = ([2, 0], [0, 2])
sets = [
    Singleton((0, 0)),
    Ellipsoid((1.5, 1), A),
    Ellipsoid((2, 2.5), A),
    Ellipsoid((3, 5), A),
    Ellipsoid((4, 7), A),
    Ellipsoid((6, 5), A),
    Ellipsoid((7, 3.5), A),
    Ellipsoid((8, 2), A),
]

# add convex sets to the graph
G = GraphOfConvexSets()
G.add_sets(sets)
G.set_source(0)
G.set_target(7)

# edges
H = np.hstack((np.eye(2), -np.eye(2)))
l = TwoNorm(H)
for u in range(len(sets)):
    for v in range(len(sets)):
        if u != v:
            G.add_edge(u, v, l)
        
# draw convex sets and edges
plt.figure()
G.draw_sets()
G.label_sets()
plt.xlim([-.3, 9])
plt.ylim([-.3, 8])
plt.grid()

In [ ]:
# add sensory-coverage constraints
spp = ShortestPathProblem(G, relaxation=1)
for v, Xv in G.sets.items():
    if v not in [0, 7]:
        Ein = G.incoming_edges(v)[1]
        spp.prog.AddLinearConstraint(sum(spp.vars.phi[Ein]) == 1)
        
# subtour elimination
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
for subtour in powerset(G.vertices[1:-1]):
    if len(subtour) >= 2:
        length = 0
        for u in subtour:
            for v in subtour:
                if u != v:
                    length += spp.vars.phi[G.edges.index((u, v))]
        spp.prog.AddLinearConstraint(length <= len(subtour) - 1)
        
    # spatial subtour elimination
    if len(subtour) == 2:
        
        u, v = subtour
        uv = G.edges.index((u, v))
        vu = G.edges.index((v, u))
        nonnegative = 1 - spp.vars.phi[uv] - spp.vars.phi[vu]
        
        Euout = G.outgoing_edges(u)[1]
        xu = sum(spp.vars.y[e] for e in Euout)
        spatial = xu - spp.vars.y[uv] - spp.vars.z[vu]
        G.sets[u].add_perspective_constraint(spp.prog, nonnegative, spatial)
        
        Evin = G.incoming_edges(v)[1]
        xv = sum(spp.vars.z[e] for e in Evin)
        spatial = xv - spp.vars.y[vu] - spp.vars.z[uv]
        G.sets[v].add_perspective_constraint(spp.prog, nonnegative, spatial)
    
sol = spp.solve()

In [ ]:
print('Cost:', sol.cost)
print('\nFlows:')
for k, edge in enumerate(G.edges):
    flow = round(abs(sol.primal.phi[k]), 4)
    print(edge, flow)

In [ ]:
plt.figure()
G.draw_sets()
G.label_sets()
G.draw_path(sol.primal.phi, sol.primal.x, color='r', linestyle='--')

plt.xticks(range(9))
plt.xlim([-.3, 9])
plt.ylim([-.3, 8])
plt.grid()
plt.savefig('sensory_coverage.pdf', bbox_inches='tight')